# 9-2

Bombillas donde duración está distribuida de forma aproximadamente normal y $\sigma = 40$ horas. En una muestra de 30 bombillas tiene una duración promedio de 78 horas. Calcular intervalo de confianza del 96% para la media de la población.

- $\sigma = 40$
- $\mu = 780$



Aquí hay un ejemplo super relevante de una función en Python que calcula intervalos de confianza

In [55]:
import statistics

def confidence_interval_old(data, confidence=0.95):
  dist = statistics.NormalDist.from_samples(data)
  z = statistics.NormalDist().inv_cdf((1 + confidence) / 2.)
  h = dist.stdev * z / ((len(data) - 1) ** .5)
  return dist.mean - h, dist.mean + h

Creates a NormalDist object from the data sample (NormalDist.from_samples(data), which gives us access to the sample's mean and standard deviation via NormalDist.mean and NormalDist.stdev.

Compute the Z-score based on the standard normal distribution (represented by NormalDist()) for the given confidence using the inverse of the cumulative distribution function (inv_cdf).

Produces the confidence interval based on the sample's standard deviation and mean.

$$\large \hat \theta_L = \bar x - z_{\alpha/2}\frac{\sigma}{\sqrt{n}}$$ y $$\large \hat \theta_U = \bar x + z_{\alpha/2}\frac{\sigma}{\sqrt{n}}$$

Haremos uso de la nueva version de Python, que incluye de manera nativa el módulo statistics. Específicamente la clase statistics.NormalDist y el método inv_cdf.

- inv_cdf(p)
  Compute the inverse cumulative distribution function, also known as the quantile function or the percent-point function. Mathematically, it is written x : P(X <= x) = p. Finds the value x of the random variable X such that the probability of the variable being less than or equal to that value equals the given probability p.

In [56]:
dist=statistics.NormalDist(mu=780, sigma=40)
dist

NormalDist(mu=780.0, sigma=40.0)

Tenemos instanciada nuestra distribución. Ahora devemos encontrar Z usando el inverso de la función de distribución. En nuestro caso nos intersa el 96% de intervalo de confianza, por lo que queremos encontrar inv_cdf(p):

$$ x : P(X <= x)=p$$

En nuestro caso p será igual a 

$$\frac{1+.96}{2} = 0.98 = 1 - \alpha/2$$

Es decir el valor $x$ de $X$ tal que la probabilidad de que $X <= x$ sea de 0.98 (Es decir 2% antes de llegar a cien, ya que tenemos dos colas en nuestra distribución) 

In [57]:
z = statistics.NormalDist().inv_cdf((1+.96) / 2.)
z

2.053748910631822

Vamos a mejorar la funcion que encontramos, dando la opcion de pasar una distribucion ya instanciada y ajustándolo a nuestra definición de $1/\sqrt(n)$ 

In [65]:
import typing


def confidence_interval(
    data: typing.Iterable = None,
    confidence: float = 0.95,
    dist: statistics.NormalDist = None,
    n: int = None,
):
    """Calculate confidence intervals with or without instanciated distributions"""
    if data is not None:
        dist = statistics.NormalDist.from_samples(data)
        z = statistics.NormalDist().inv_cdf((1 + confidence) / 2.0)
        h = dist.stdev * z / ((len(data)) ** 0.5)
        return dist.mean - h, dist.mean + h
    else:
        z = statistics.NormalDist().inv_cdf((1 + confidence) / 2.0)
        h = dist.stdev * z / ((n) ** 0.5)
        return dist.mean - h, dist.mean + h


In [66]:
confidence_interval(confidence=0.96,dist=dist,n=30)

(765.0015385892706, 794.9984614107294)

Es decir:

 $$ 765.0015 < \mu <794.998 $$
 
 con 96% de confianza 

# 9.3
 
Calcular un intevalo de confianza del 95% dado:
$$\sigma = 0.0015$$
$$n=75$$
$$\mu = 0.310$$

Tomando en cuenta que ya tenemos una función creada en el inciso anterior, podemos simplemente utilizarla para obtener el intevalo de confianza

In [67]:
dist=statistics.NormalDist(mu=0.310, sigma=0.0015)
dist

NormalDist(mu=0.31, sigma=0.0015)

In [68]:
confidence_interval(confidence=0.95,dist=dist,n=75)

(0.30966052427977714, 0.31033947572022286)

Es decir:

 $$ 0.30966 < \mu < 0.3103 $$
 
 con 95% de confianza 

# 9.6 

¿Qué tan grande debe ser la muestra en el ejercicio 9.2 si deseamos tener un 96% de confianza en que nuestra media muestral estará dentro de 10 horas a partir de la media verdadera?

$$n = \Big(\frac{z_{\alpha/2} \sigma}{e}\Big)^2 $$

Recordande que 

- $\sigma = 40$
- $\mu = 780$

Primeramente calcularemos nuevamente $z_{\alpha/2}$

In [69]:
z = statistics.NormalDist().inv_cdf((1+.96) / 2.)
z

2.053748910631822

In [70]:
n = (z*40 / 10)**2
n

67.48615340674233

Redondeando, obtenemos la respuesta de $n=68$

# 9.7

Similar al ejercicio anterior, tenemos ahora que $e=0.0005$ con una confianza del 95%

Primeramente calcularemos nuevamente $z_{\alpha/2}$

$$\sigma = 0.0015$$
$$n=75$$
$$\mu = 0.310$$

In [71]:
z = statistics.NormalDist().inv_cdf((1+.95) / 2.)
z

1.9599639845400536

In [72]:
n = (z*0.0015/0.0005)**2
n

34.57312938624712

Por lo que al redondear, tenemos la respuesta $n=35$

# 9.14



In [73]:
import pandas as pd
df=pd.read_csv("914.csv")
df

,data
0,3.4
1,2.5
2,4.8
3,2.9
4,3.6
5,2.8
6,3.3
7,5.6
8,3.7
9,2.8


In [80]:
df.describe()

,data
count,15.000000
mean,3.786667
std,0.970910
min,2.500000
25%,2.950000
50%,3.600000
75%,4.600000
max,5.600000


In [75]:
confidence_interval(data=df["data"],confidence=0.95)

(3.295327328283576, 4.278006005049757)